In [7]:
import os
import time
import csv
import re
import codecs
import contextlib
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException

## setup area

In [8]:
read_filename = "restaurant_names.txt"
write_filename = "review1.csv"

read_path =os.path.join("response", read_filename)
write_path = os.path.join("review",write_filename)

In [9]:
list_kategori = ['opsi_layanan','keunggulan','aksesbilitas','penawaran',
                'pilihan_makanan','fasilitas','suasana','tipe_pengunjung',
                'perencanaan','pembayaran']

In [10]:
# Initialize empty lists for each category
opsi_layanan = []
keunggulan = []
aksesbilitas = []
penawaran = []
pilihan_makanan = []
fasilitas = []
suasana = []
tipe_pengunjung = []
perencanaan = []
pembayaran = []

In [11]:
number = 2

## Google maps function

In [12]:
def search(name):
    global driver , actions
    global link
    
    link = "https://www.google.com/maps/search/{} jogja".format(name)
    
    #open up chrome and stuff
    driver = webdriver.Chrome()
    driver.get(link)
    actions = ActionChains(driver)
    
    #kalau gak langsung ketemu
    
    try:
        restaurant_link = driver.find_elements(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a')
        
        for links in restaurant_link:
            link = links.get_attribute("href")
            driver.get(link)
    except:
        pass
        
    
    print('starting scraping for:{}'.format(name))

In [13]:
def about_button():
    about_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]')))
    actions.move_to_element(about_button)
    actions.click().perform()

## scrape about function

In [18]:
# def scrape_list():
#     delay = 5
#     #opsi layanan
#     try:
#         list_element = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 opsi_layanan.append(point_element.text)
    
#     except:
#         opsi_layanan.append('-')
        

#     #keunggulan
#     try:
#         list_element = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[5]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 keunggulan.append(point_element.text)
    
#     except:
#         keunggulan.append('-')
        
#     #aksesbilitas
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 aksesbilitas.append(point_element.text)
    
#     except:
#         aksesbilitas.append('-')
        
#     #penawaran
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[11]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 penawaran.append(point_element.text)
    
#     except:
#         penawaran.append('-')
        
#     #pilihan makanan
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[13]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 piliihan_makanan.append(point_element.text)
    
#     except:
#         pilihan_makanan.append('-')
        
#     #fasilitas
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[15]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 fasilitas.append(point_element.text)
    
#     except:
#         fasilitas.append('-')
        
#     #suasana
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[18]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 suasana.append(point_element.text)
    
#     except:
#         suasana.append('-')
        
#     #tipe pengunjung
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[21]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 tipe_pengunjung.append(point_element.text)
    
#     except:
#         tipe_pengunjung.append('-')
        
#     #perencanaan
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[24]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 perencanaan.append(point_element.text)
    
#     except:
#         perencanaan.append('-')
        
#     #pembayaran
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[27]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 pembayaran.append(point_element.text)
    
#     except:
#         pembayaran.append('-')
        
    

In [41]:
def scrape_list(xpath):
    global points , number
    points = []
    
    try:
        list_element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath.format(number))))
        # Extract all the points from the list
        point_elements = list_element.find_elements(By.TAG_NAME, "li")

        # Check the class of each point and append the points with the specified class
        for point_element in point_elements:
            point_class = point_element.get_attribute("class")
            if point_class == "hpLkke ":
                points.append(point_element.text)
    
    except:
        points.append('-')
        
    number +=3

In [42]:
list_kategori = ['opsi_layanan','keunggulan','aksesbilitas','penawaran',
                'pilihan_makanan','fasilitas','suasana','tipe_pengunjung',
                'perencanaan','pembayaran']

In [44]:
def scrape_about():
    global deskripsi , number
    global opsi_layanan , keunggulan , aksesbilitas
    global penawaran , pilihan_makanan , fasilitas
    global suasana , tipe_pengunjung , perencanaan
    global pembayaran
    number = 2
    xpath_deskripsi = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[{}]/p'.format(number)
    xpath_feature = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[{}]/ul'
    deskripsi = '-'
    
    scrape_list(xpath_feature)
    opsi_layanan = points

    scrape_list(xpath_feature)
    keunggulan = points
    
    scrape_list(xpath_feature)
    aksesbilitas = points
    
    scrape_list(xpath_feature)
    penawaran = points
    
    scrape_list(xpath_feature)
    pilihan_makanan = points
    
    scrape_list(xpath_feature)
    fasilitas = points
    
    scrape_list(xpath_feature)
    suasana = points
    
    scrape_list(xpath_feature)
    tipe_pengunjung = points
    
    scrape_list(xpath_feature)
    perencanaan = points

    scrape_list(xpath_feature)
    pembayaran = points
    
    
    try:
        raw_deskripsi = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath_deskripsi)))
        deskripsi = raw_deskripsi.text
    
    except:
        pass
    


In [40]:
#Test cell

test ="'D'coff jogja"
search(test)
about_button()
scrape_about()

starting scraping for:'D'coff jogja
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[5]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[11]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[14]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[17]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[20]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[23]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[26]/ul
/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[29]/ul
['Tempat duduk di area terbuka', 'Ambil di tepi jalan', '

### setup csv file

In [47]:
# Define the fieldnames for the CSV file
fieldnames = ['DESCRIPTION','SERVICE OPTION','HIGHLIGHTS' , 'ACCESSIBILITY' , 'OFFERINGS' , 'DINING OPTIONS' , 'AMENITIES' , 'ATHMOSPHERE', 'CROWD', 'PLANNING', 'PAYMENTS']

In [50]:
driver = webdriver.Chrome()
# Create the 'review' directory if it doesn't exist
os.makedirs('feature', exist_ok=True)

# Open the input file and loop through each vessel name
with open(read_path, newline='') as csvfile:
    reader = csv.reader(csvfile)

    next(reader)
    for row in reader:
        NAMA = row[0]
        nomor = 1
        review_count = 0

        # Generate the write file path based on the restaurant name
        write_filename = f"{NAMA}.csv"
        write_path = os.path.join("feature", write_filename)
        
        # Check if the file already exists and have more than 20 lines
        if os.path.exists(write_path):
            with open(write_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                non_empty_lines = [line for line in lines if line.strip()]  # Filter non-empty lines
                if len(non_empty_lines) > 1:
                    print(f"{NAMA} is already scrapped. Check or delete the file if you want to rescrape it.")
                    driver.quit()  # Close the webdriver
                    continue
        try:
            search(NAMA)
        except:
            continue
            
            
        about_button()
        
        scrape_about()
        
        details_dict = {
            'DESCRIPTION': deskripsi,
            'SERVICE OPTION': opsi_layanan,
            'HIGHLIGHTS': keunggulan,
            'ACCESSIBILITY': aksesbilitas,
            'OFFERINGS': penawaran,
            'DINING OPTIONS': pilihan_makanan,
            'AMENITIES': fasilitas,
            'ATHMOSPHERE': suasana,
            'CROWD': tipe_pengunjung,
            'PLANNING': perencanaan,
            'PAYMENTS': pembayaran
        }
        
        # Determine the maximum length among all lists
        max_length = max(len(value) if isinstance(value, list) else 1 for value in details_dict.values())
        
        # Open the CSV file for writing with UTF-8 encoding
        with codecs.open(write_path, mode='w', encoding='utf-8', errors='ignore') as details_file:
            writer = csv.writer(details_file)
            writer.writerow(details_dict.keys())
            
            # Write the data to the CSV file row-wise
            for i in range(max_length):
                row_data = {}
                for key, value in details_dict.items():
                    if isinstance(value, list) and i < len(value):
                        row_data[key] = value[i]
                    elif not isinstance(value, list) and i == 0:
                        row_data[key] = value
                    else:
                        row_data[key] = ''
                
                writer.writerow(row_data.values())

        print(f"feature scraping finished for {NAMA}")

print('done!')
driver.quit()  # Close the webdriver

starting scraping for:Roaster And Bear Restaurant
feature scraping finished for Roaster And Bear Restaurant
starting scraping for:Kalluna - Instagrammable Restaurant & Meeting Room
feature scraping finished for Kalluna - Instagrammable Restaurant & Meeting Room
starting scraping for:Secret Garden Coffee and Chocolate
feature scraping finished for Secret Garden Coffee and Chocolate
starting scraping for:Water castle cafe
feature scraping finished for Water castle cafe
starting scraping for:DOUBLE O Resto and Cafe
feature scraping finished for DOUBLE O Resto and Cafe
starting scraping for:Lecker Rumah Kopi & Resto
feature scraping finished for Lecker Rumah Kopi & Resto
starting scraping for:POENOKAWAN CAFE RESTO & GALLERY
feature scraping finished for POENOKAWAN CAFE RESTO & GALLERY
starting scraping for:Palms Resto & Cafe
feature scraping finished for Palms Resto & Cafe
starting scraping for:Warung Lawas
feature scraping finished for Warung Lawas
starting scraping for:Atap Cafe & Resto
